In [3]:
import pandas as pd
import os
import string 
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from datetime import datetime

In [4]:
q = """ SELECT * FROM `wagon-bootcamp-328013.trip.comment`"""
data = pd.read_gbq(q,project_id="wagon-bootcamp-328013")

/Users/nicolasmanoharan/.pyenv/versions/3.8.12/envs/FOBO/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2009: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


In [5]:
data

,date,note,titre,commentaire,adresse,nom
0,2021-07-27 00:00:00+00:00,2.0,Cuisine médiocre à prix trop élevé,Nos plats étaient de qualité médiocre et nous ...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO
1,2016-02-17 00:00:00+00:00,2.0,complètement surfait,le type de restau qui est une pure tromperie s...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO
2,2015-03-13 00:00:00+00:00,2.0,Le prix ... et la mentalité....,Restaurant a la décoration classique assez étr...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO
3,2020-10-02 00:00:00+00:00,3.0,Décevant,"Salle en bas très bruyante, la musique est tro...","10 Rue de Port-Mahon, 75002 Paris France",14. YO
4,2018-12-11 00:00:00+00:00,3.0,Très bon service mais un peu bruyant,Très bon accueil et très bon service dans ce r...,"10 Rue de Port-Mahon, 75002 Paris France",14. YO
...,...,...,...,...,...,...
638785,2017-11-11 00:00:00+00:00,5.0,Magnifique,Nous avons voulu tester pour la première fois....,"8 Rue du Vieux Colombier, 75006 Paris France",354. Christophe Michalak - Saint-Germain des Prés
638786,2017-07-11 00:00:00+00:00,5.0,Exceptionnel.... Gâteaux renversants.,Gâteaux individuels ou en 6-8 parts sont à cou...,"8 Rue du Vieux Colombier, 75006 Paris France",354. Christophe Michalak - Saint-Germain des Prés
638787,2017-07-09 00:00:00+00:00,5.0,Genial,"De super pâtisseries à tomber , la verine pist...","8 Rue du Vieux Colombier, 75006 Paris France",354. Christophe Michalak - Saint-Germain des Prés
638788,2018-11-21 00:00:00+00:00,1.0,1ere fois,"Pur produit pour touristes, accueil pompeux/sn...","8 Rue du Vieux Colombier, 75006 Paris France",354. Christophe Michalak - Saint-Germain des Prés


In [6]:
def cleaner(data):
    data['review_clean'] = data['commentaire'].apply(clear_data_text)
    #data['date'] = data['date'].apply(change_date)
    #data['rate'] = data['rate'].apply(keep_digit)
    data['review_sentences'] = data['review_clean'].apply(splitter)
    df_exploded = data.explode('review_sentences').reset_index(drop=True)
    return df_exploded

def splitter(text):
    tmp = re.split('[.?!]', text)
    tmp = filter(None, tmp)
    return list(tmp)


def clear_data_text(text, language='french',deep_clean=False):
    """
    Returns a clean text as a string. Default language = english.
    Cleaning includes : removing punct, digit and \\n, stopwords, informations
    related to author review, and lowercase.
    """
    #remove before the begin of the comment
    for i in range(5):
        pos = 0
        pos = text.find('\n')
        text = text[pos + 1:]

    text=text.replace('\n',' ')
    text=text.replace('/','')

    if deep_clean:
        #remove punctuation
        for punctuation in string.punctuation:
            text = text.replace(punctuation, '')

    if True:
        # remove number
        text = ''.join(word for word in text if not word.isdigit())

    #pass in lowercase
    text = text.lower()

    if deep_clean:
        #remove stop words
        stop_words = set(stopwords.words(language))

        #text to list of word
        word_tokens = word_tokenize(text)

        #list to string
        text = " ".join([w for w in word_tokens if not w in stop_words])

    #remove after the end of the comment
    text=text.strip()
    text=text.replace('  ',' ')
    if text.endswith('useful funny cool'):
        text = text[:-len('useful funny cool')]
        text = text.strip()
    if text.startswith('updated review'):
        text = text.split('useful funny cool', 1)[0]
        text = text[len('updated review'):]
        text = text.strip()
    if text.startswith('photos'):
        text = text[len('photos'):]
        text = text.strip()
    if text.startswith('photo'):
        text = text[len('photo'):]
        text = text.strip()
    text = text.strip()
    return text


In [7]:
data['review_clean'] = data['commentaire'].apply(clear_data_text)
data['review_sentences'] = data['review_clean'].apply(splitter)
df_exploded = data.explode('review_sentences').reset_index(drop=True)

In [12]:
sample = df_exploded.head(100000)

In [9]:
from sentence_transformers import SentenceTransformer, util


model = SentenceTransformer('symanto/sn-xlm-roberta-base-snli-mnli-anli-xnli')

In [13]:
%%time
sample = [i for i in sample["review_sentences"] if len(i) < 500]



embeddings = model.encode(sample)

CPU times: user 2h 57min 20s, sys: 52.2 s, total: 2h 58min 12s
Wall time: 2h 51min 58s


In [11]:
import pytorch

ModuleNotFoundError: No module named 'pytorch'

In [ ]:
model